In [8]:
# imports
import numpy as np
import os, sys, gc
from scipy.spatial.distance import mahalanobis
import matplotlib.pyplot as plt

from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation
from keras import backend as K

In [9]:
emprical_means_h = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_mahalanobis_training_samples_emprical_means_h.npy')
emprical_means_a = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_mahalanobis_training_samples_emprical_means_a.npy')
emprical_means_n = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_mahalanobis_training_samples_emprical_means_n.npy')
emprical_means_s = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_mahalanobis_training_samples_emprical_means_s.npy')
emprical_means_o = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_mahalanobis_training_samples_emprical_means_o.npy')

In [10]:
emprical_std_means_h = np.std(emprical_means_h)
emprical_std_means_a = np.std(emprical_means_a)
emprical_std_means_n = np.std(emprical_means_n)
emprical_std_means_s = np.std(emprical_means_s)
emprical_std_means_o = np.std(emprical_means_o)

emprical_mean_of_means_h = np.mean(emprical_means_h)
emprical_mean_of_means_a = np.mean(emprical_means_a)
emprical_mean_of_means_n = np.mean(emprical_means_n)
emprical_mean_of_means_s = np.mean(emprical_means_s)
emprical_mean_of_means_o = np.mean(emprical_means_o)

In [15]:
stds_five = [emprical_std_means_h, emprical_std_means_a, emprical_std_means_n, emprical_std_means_s, emprical_std_means_o]
means_five = [emprical_mean_of_means_h, emprical_mean_of_means_a, emprical_mean_of_means_n, emprical_mean_of_means_s, emprical_mean_of_means_o]

In [11]:
mius = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_emprical_means.npy')
covariance_mat = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_emprical_covariance.npy')

In [12]:
#C = [0, 1, 2, 3, 4]
#maha_scores_calm = calc_mahalanobis(featureSet_calm, Label_calm, C, covariance_mat, mius)
#np.save('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_calm_maha_scores.npy', maha_scores_calm)
maha_scores_calm = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_calm_maha_scores.npy', allow_pickle=True)

In [13]:
#C = [0, 1, 2, 3, 4]
#maha_scores_surprise = calc_mahalanobis(featureSet_surprise, Label_surprise, C, covariance_mat, mius)
#np.save('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_surprise_maha_scores.npy', maha_scores_surprise)
maha_scores_surprise = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_surprise_maha_scores.npy', allow_pickle=True)

In [27]:
maha_scores_test = np.load('..//..//modules//Checkpoint_home_All_neurons_home_4096_filters_3072_dropout_0.2_epoch_1000_dense_10_penultimate_m_scores_norm_test_set_no_calm_or_surprise.npy')

In [14]:
def min_index(a): 
    a = list(a)
    # inbuilt function to find the position of minimum  
    minpos = a.index(min(a))  
    # inbuilt function to find the position of maximum  
    maxpos = a.index(max(a))    
    return minpos

def max_index(a): 
    a = list(a)
    # inbuilt function to find the position of minimum  
    minpos = a.index(min(a))  
    # inbuilt function to find the position of maximum  
    maxpos = a.index(max(a))    
    return maxpos

In [89]:
def test_ood_samples(maha_scores_vector, means_for_all_classes, stds_for_all_classes):
    ind = 0
    ood = 0
    
    for item in maha_scores_vector:
        
        if np.array(item).shape != (5,):
            rearranged = [item[0][0], item[1][0], item[2][0], item[3][0], item[4][0]]
        else:
            rearranged = item
        plausible = min_index(rearranged)
    
    if means_for_all_classes[plausible] - 1.5 * stds_for_all_classes[plausible] < rearranged[plausible] and rearranged[plausible] < means_for_all_classes[plausible] + 1.5 * stds_for_all_classes[plausible]:
        ind += 1
    else:
        ood += 1
    
    return ind, ood

In [90]:
ind_calm, ood_calm = test_ood_samples(maha_scores_calm, means_five, stds_five)
print('Accuracy of the ood algorithm on calm samples are: ' + str(ood_calm/(ind_calm+ood_calm)) )

Accuracy of the ood algorithm on calm samples are: 1.0


In [91]:
ind_surprise, ood_surprise = test_ood_samples(maha_scores_surprise, means_five, stds_five)
print('Accuracy of the ood algorithm on surprise samples are: ' + str(ood_surprise/(ind_surprise+ood_surprise)) )

Accuracy of the ood algorithm on surprise samples are: 0.0


In [92]:
ind_test, ood_test = test_ood_samples(maha_scores_test, means_five, stds_five)
print('Accuracy of the ood algorithm on calm samples are: ' + str(ind_test/(ind_test+ood_test)) )

Accuracy of the ood algorithm on calm samples are: 1.0
